## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [10]:
from keras import backend as K
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
#Blas GEMM launch failed , 避免動態分配GPU / CPU, 出現問題
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [12]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [13]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練的 epochs 數量

In [14]:
# 建立 ResNet 模型
K.clear_session() # 把舊的 Graph 清掉
model = ResNet50(weights='imagenet', include_top=False,input_shape=(32,32,3))
x = model.output
x = Flatten()(x)
x = Dropout(0.25)(x)
output = Dense(num_classes, activation='softmax', name='softmax')(x)
net_final = Model(inputs=model.input, outputs=output)
net_final.summary()

C:\ProgramData\Anaconda3\envs\tf_2.1\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 96s 1us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________

In [15]:
#compile
net_final.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy', metrics=['accuracy'])
#ImageDataGenerator
augment_generator = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

history = net_final.fit_generator(
                    augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/30
781/781 [==============================] - 95s 121ms/step - loss: 2.5849 - accuracy: 0.1177 - val_loss: 4.1348 - val_accuracy: 0.1141
Epoch 2/30
781/781 [==============================] - 84s 108ms/step - loss: 2.4111 - accuracy: 0.1253 - val_loss: 5.1303 - val_accuracy: 0.1780
Epoch 3/30
781/781 [==============================] - 85s 109ms/step - loss: 2.3490 - accuracy: 0.1344 - val_loss: 3.3573 - val_accuracy: 0.2087
Epoch 4/30
781/781 [==============================] - 85s 109ms/step - loss: 2.3231 - accuracy: 0.1324 - val_loss: 3.4596 - val_accuracy: 0.2130
Epoch 5/30
781/781 [==============================] - 85s 109ms/step - loss: 2.3036 - accuracy: 0.1327 - val_loss: 3.0668 - val_accuracy: 0.2238
Epoch 6/30
781/781 [==============================] - 86s 110ms/step - loss: 2.2893 - accuracy: 0.1374 - val_loss: 3.0436 - val_accuracy: 0.2444
Epoch 7/30
781/781 [==============================] - 86s 110ms/step - loss: 2.2742 - accuracy: 0.1395 - val_loss: 3.0515 - val_ac

In [16]:
score = net_final.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 5s 534us/step
Test loss: 2.2929665981292726
Test accuracy: 0.4745999872684479
